# Generierung synthetischer Prompts für Gerichtsurteile und Pressemitteilungen

Dieses Notebook verwendet die Claude API, um synthetische Prompts für die Pressemitteilungen - Gerichtsurteilen Paare zu erstellen.

## 1. Setup und Datensatz-Laden

Zuerst importieren wir die notwendigen Bibliotheken und laden den bereinigten Datensatz aus `bereinigung.ipynb`.

In [ ]:
import pandas as pd
import numpy as np
import os
import requests
import json
import time
from tqdm.auto import tqdm
import anthropic
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Lade den bereinigten Datensatz
try:
    df = pd.read_csv('cleaned_data/cleaned_combined_methods.csv')
    print(f"Bereinigter Datensatz mit {len(df)} Einträgen geladen")
except FileNotFoundError:
    print("Bereinigter Datensatz nicht gefunden. Bitte führe zuerst bereinigung.ipynb aus.")
    # Für Demonstrationszwecke fahren wir fort, als ob der Datensatz existieren würde

# Stichprobe des Datensatzes für Tests (für Produktionseinsatz auskommentiert)
# df = df.sample(5, random_state=42)

## 2. Setup Claude API-Zugang

Als nächstes richten wir die Verbindung zur Claude API ein. Wir verwenden den Anthropic Python-Client.

In [ ]:
# Setup des Claude API-Clients
# Besorge deinen API-Schlüssel von https://console.anthropic.com/
# Setze deinen API-Schlüssel als Umgebungsvariable oder direkt hier (nicht empfohlen für geteilte Notebooks)

ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")

if not ANTHROPIC_API_KEY:
    ANTHROPIC_API_KEY = input("Bitte gib deinen Anthropic API-Schlüssel ein: ")
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

# Initialisiere den Claude-Client
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

# Teste die API-Verbindung
try:
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=100,
        messages=[
            {"role": "user", "content": "Hallo, Claude! Bitte antworte mit einer sehr kurzen Begrüßung."}
        ]
    )
    print("API-Verbindung erfolgreich!")
    print(f"Claude sagt: {response.content[0].text}")
except Exception as e:
    print(f"Fehler bei der Verbindung zur Claude API: {e}")

## 3. Hilfsfunktionen für die Generierung synthetischer Prompts

Wir erstellen Hilfsfunktionen, um synthetische Prompts aus den Gerichtsurteilen und Pressemitteilungen zu generieren.

In [ ]:
def generate_synthetic_prompt(court_ruling, press_release, model="claude-3-haiku-20240307", retries=3, wait_time=2):
    """
    Generiert einen synthetischen Prompt, der die gegebene Pressemitteilung aus dem Gerichtsurteil erzeugen könnte.
    
    Args:
        court_ruling (str): Der Text des Gerichtsurteils
        press_release (str): Der Text der Pressemitteilung
        model (str): Das zu verwendende Claude-Modell
        retries (int): Anzahl der Wiederholungsversuche bei API-Fehlern
        wait_time (int): Wartezeit zwischen Wiederholungsversuchen in Sekunden
        
    Returns:
        str: Der generierte synthetische Prompt
    """
    # Vorbereitung des System-Prompts
    system_prompt = """
    Du bist ein Experte für juristische Texte und Kommunikation. Deine Aufgabe ist es, ein Gerichtsurteil und die 
    dazugehörige Pressemitteilung zu analysieren und dann herauszufinden, welcher Prompt verwendet worden sein könnte, 
    um diese Pressemitteilung aus dem Gerichtsurteil zu generieren, wenn man ihn einem LLM gegeben hätte.
    
    1. Analysiere, wie die Pressemitteilung Informationen aus dem Urteil vereinfacht, umstrukturiert und Schlüsselinformationen hervorhebt
    2. Berücksichtige den Ton, die Struktur und den Detaillierungsgrad der Pressemitteilung
    3. Identifiziere, welche Anweisungen nötig wären, um den juristischen Text in diese Pressemitteilung zu transformieren
    
    Erkläre NICHT deine Überlegungen und füge KEINE Meta-Kommentare hinzu. Gib NUR den tatsächlichen Prompt aus, der die 
    Pressemitteilung aus dem Gerichtsurteil generieren würde. Sei spezifisch und detailliert in deinem synthetisierten Prompt.
    """
    
    # Vorbereitung des Benutzer-Prompts
    user_prompt = f"""
    Hier ist das originale Gerichtsurteil:
    
    ```
    {court_ruling}
    ```
    
    Und hier ist die Pressemitteilung, die daraus erstellt wurde:
    
    ```
    {press_release}
    ```
    
    Erstelle einen detaillierten Prompt, der einem LLM gegeben werden könnte, um die obige Pressemitteilung aus dem Gerichtsurteil zu generieren. 
    Schreibe NUR den Prompt selbst, ohne Erklärungen oder Meta-Kommentare.
    """
    
    for attempt in range(retries):
        try:
            response = client.messages.create(
                model=model,
                max_tokens=1000,
                system=system_prompt,
                messages=[
                    {"role": "user", "content": user_prompt}
                ]
            )
            return response.content[0].text.strip()
        except Exception as e:
            if attempt < retries - 1:
                print(f"Fehler: {e}. Neuer Versuch in {wait_time} Sekunden...")
                time.sleep(wait_time)
            else:
                print(f"Nach {retries} Versuchen fehlgeschlagen: {e}")
                return f"Fehler bei der Generierung des Prompts: {e}"

def process_batch(df, batch_size=10, start_idx=0, save_interval=10):
    """
    Verarbeitet einen Dataframe in Batches und generiert synthetische Prompts für jede Zeile.
    
    Args:
        df (pd.DataFrame): Der Dataframe mit Gerichtsurteilen und Pressemitteilungen
        batch_size (int): Anzahl der Elemente, die vor dem Speichern von Zwischenergebnissen verarbeitet werden
        start_idx (int): Index, bei dem die Verarbeitung beginnen soll (für die Wiederaufnahme)
        save_interval (int): Wie oft Zwischenergebnisse gespeichert werden sollen
        
    Returns:
        pd.DataFrame: Der Dataframe mit hinzugefügten synthetischen Prompts
    """
    # Erstelle eine synthetic_prompt-Spalte, falls sie nicht existiert
    if 'synthetic_prompt' not in df.columns:
        df['synthetic_prompt'] = None
    
    # Erstelle ein Verzeichnis für Checkpoints, falls es nicht existiert
    checkpoint_dir = Path('checkpoints')
    checkpoint_dir.mkdir(exist_ok=True)
    
    # Verarbeite in Batches
    for i in tqdm(range(start_idx, len(df), batch_size)):
        batch_end = min(i + batch_size, len(df))
        batch = df.iloc[i:batch_end].copy()
        
        for idx, row in batch.iterrows():
            if pd.isna(df.at[idx, 'synthetic_prompt']):
                court_ruling = row['judgement']
                press_release = row['summary']
                
                # Kürze sehr lange Eingaben, um Token-Limits zu vermeiden
                if len(court_ruling) > 15000:
                    court_ruling = court_ruling[:15000] + "..."
                if len(press_release) > 5000:
                    press_release = press_release[:5000] + "..."
                
                # Generiere den synthetischen Prompt
                synthetic_prompt = generate_synthetic_prompt(court_ruling, press_release)
                df.at[idx, 'synthetic_prompt'] = synthetic_prompt
                
                # Zeige Fortschritt an
                if (idx - i) % 5 == 0 or idx == batch_end - 1:
                    print(f"{idx+1}/{len(df)} Einträge verarbeitet")
        
        # Speichere Checkpoint in regelmäßigen Abständen
        if (i // batch_size) % save_interval == 0 or batch_end == len(df):
            checkpoint_path = checkpoint_dir / f"synthetic_prompts_checkpoint_{batch_end}.csv"
            df.to_csv(checkpoint_path, index=False)
            print(f"Checkpoint gespeichert unter {checkpoint_path}")
    
    return df

## 4. Checkpoint laden oder von vorne beginnen

Wir prüfen, ob es eine Checkpoint-Datei gibt, um von dort fortzufahren, oder beginnen die Verarbeitung von Anfang an.

In [ ]:
# Prüfe auf vorhandene Checkpoints
checkpoint_dir = Path('checkpoints')
checkpoint_files = list(checkpoint_dir.glob("synthetic_prompts_checkpoint_*.csv")) if checkpoint_dir.exists() else []

start_idx = 0

if checkpoint_files:
    # Finde den neuesten Checkpoint basierend auf der Nummer im Dateinamen
    latest_checkpoint = max(checkpoint_files, key=lambda x: int(x.stem.split('_')[-1]))
    print(f"Neuester Checkpoint gefunden: {latest_checkpoint}")
    
    # Frage, ob vom Checkpoint fortgefahren werden soll
    continue_from_checkpoint = input("Vom letzten Checkpoint fortfahren? (j/n): ").lower().strip() == 'j'
    
    if continue_from_checkpoint:
        # Lade den Checkpoint
        df = pd.read_csv(latest_checkpoint)
        print(f"Checkpoint mit {len(df)} Einträgen geladen")
        
        # Finde die erste Zeile ohne synthetischen Prompt
        if 'synthetic_prompt' in df.columns:
            missing_prompts = df['synthetic_prompt'].isna()
            if missing_prompts.any():
                start_idx = missing_prompts.idxmax()
                print(f"Verarbeitung wird ab Index {start_idx} fortgesetzt")
            else:
                print("Alle Einträge haben bereits synthetische Prompts!")
        else:
            print("Keine synthetic_prompt-Spalte im Checkpoint gefunden, beginne von vorne")
    else:
        print("Starte neue Verarbeitung")

## 5. Synthetische Prompts generieren

Jetzt verarbeiten wir den Datensatz, um die synthetischen Prompts zu generieren.

In [ ]:
# Konfiguriere Parameter für die Batch-Verarbeitung
batch_size = 10  # Verarbeite 10 Einträge auf einmal
save_interval = 5  # Speichere Checkpoint alle 5 Batches

# Verarbeite den Datensatz
print(f"Starte Generierung synthetischer Prompts für {len(df)} Einträge ab Index {start_idx}")
df = process_batch(df, batch_size=batch_size, start_idx=start_idx, save_interval=save_interval)

# Speichere das endgültige Ergebnis
output_path = Path('data/court_press_with_synthetic_prompts.csv')
df.to_csv(output_path, index=False)
print(f"Vollständiger Datensatz mit synthetischen Prompts gespeichert unter {output_path}")

## 6. Beispiele der generierten synthetischen Prompts analysieren

Schauen wir uns einige Beispiele der generierten synthetischen Prompts an, um die Art der Anweisungen zu verstehen, die Claude vorgeschlagen hat.

In [ ]:
# Wähle einige Beispiele zur Anzeige aus
if 'synthetic_prompt' in df.columns and not df['synthetic_prompt'].isna().all():
    # Hole 5 zufällige Beispiele, bei denen synthetic_prompt nicht None/NaN ist
    sample_df = df[df['synthetic_prompt'].notna()].sample(min(5, len(df[df['synthetic_prompt'].notna()])))
    
    for i, (_, row) in enumerate(sample_df.iterrows(), 1):
        print(f"\n==== Beispiel {i} ====")
        print(f"Gerichtsfall-ID: {row['id']}")
        print("\nSynthetischer Prompt:")
        print(row['synthetic_prompt'])
        
        # Optional, zeige auch den ersten Teil der Pressemitteilung
        print("\nErste 150 Zeichen der Pressemitteilung:")
        print(row['summary'][:150] + "...")
        print("-" * 80)
else:
    print("Noch keine synthetischen Prompts verfügbar. Führe zunächst die Verarbeitungszelle aus.")

## 7. Muster in den synthetischen Prompts analysieren

Analysieren wir die generierten Prompts, um häufige Muster und Eigenschaften zu identifizieren.

In [ ]:
# Prüfe, ob wir Prompts zur Analyse haben
if 'synthetic_prompt' in df.columns and not df['synthetic_prompt'].isna().all():
    # Erstelle einen Dataframe nur mit den synthetischen Prompts für die Analyse
    prompts_df = df[df['synthetic_prompt'].notna()].copy()
    
    # Berechne grundlegende Statistiken
    prompt_lengths = prompts_df['synthetic_prompt'].str.len()
    avg_length = prompt_lengths.mean()
    min_length = prompt_lengths.min()
    max_length = prompt_lengths.max()
    
    print(f"Anzahl synthetischer Prompts: {len(prompts_df)}")
    print(f"Durchschnittliche Prompt-Länge: {avg_length:.1f} Zeichen")
    print(f"Kürzester Prompt: {min_length} Zeichen")
    print(f"Längster Prompt: {max_length} Zeichen")
    
    # Suche nach häufigen Phrasen oder Anweisungstypen in den Prompts
    common_phrases = [
        "zusammenfassen", "vereinfachen", "erklären", "übersetzen", "umwandeln", 
        "transformieren", "erstelle eine pressemitteilung", "schreibe eine pressemitteilung", 
        "nicht-technisch", "laien", "öffentliches publikum", "prägnant"
    ]
    
    phrase_counts = {}
    for phrase in common_phrases:
        count = prompts_df['synthetic_prompt'].str.lower().str.contains(phrase).sum()
        if count > 0:
            phrase_counts[phrase] = count
    
    # Konvertiere Anzahl in Prozentsätze
    phrase_percentages = {k: v/len(prompts_df)*100 for k, v in phrase_counts.items()}
    
    # Sortiere nach Häufigkeit
    sorted_phrases = sorted(phrase_percentages.items(), key=lambda x: x[1], reverse=True)
    
    print("\nHäufige Anweisungstypen in synthetischen Prompts:")
    for phrase, percentage in sorted_phrases:
        print(f"{phrase}: {percentage:.1f}% der Prompts")
else:
    print("Noch keine synthetischen Prompts verfügbar. Führe zunächst die Verarbeitungszelle aus.")

## 8. Datensatz für weitere Analysen exportieren

Schließlich speichern wir unseren Datensatz mit den synthetischen Prompts für weitere Analysen.

In [ ]:
# Erstelle eine endgültige Version des Datensatzes mit Spalten in einer logischen Reihenfolge
if 'synthetic_prompt' in df.columns:
    # Reorganisiere Spalten in einer logischen Reihenfolge
    columns_order = [
        'id', 'date', 'subset_name', 'split_name',  # Kennungen
        'synthetic_prompt',  # Neue generierte Spalte
        'summary', 'judgement'  # Ursprüngliche Textspalten
    ]
    
    # Füge alle verbleibenden Spalten hinzu
    remaining_columns = [col for col in df.columns if col not in columns_order]
    final_columns = columns_order + remaining_columns
    
    # Erstelle endgültigen Datensatz mit Spalten in gewünschter Reihenfolge
    final_df = df[final_columns].copy()
    
    # Speichere als endgültigen Datensatz
    final_output_path = Path('data/court_press_with_synthetic_prompts_final.csv')
    final_df.to_csv(final_output_path, index=False)
    print(f"Endgültiger Datensatz mit {len(final_df)} Einträgen gespeichert unter {final_output_path}")
    
    # Speichere auch eine leichtere Version mit nur den wesentlichen Spalten
    essential_columns = ['id', 'date', 'subset_name', 'split_name', 'synthetic_prompt', 'summary', 'judgement']
    essential_df = final_df[essential_columns].copy()
    
    essential_output_path = Path('data/court_press_with_synthetic_prompts_essential.csv')
    essential_df.to_csv(essential_output_path, index=False)
    print(f"Wesentlicher Datensatz mit {len(essential_df)} Einträgen gespeichert unter {essential_output_path}")
else:
    print("Noch keine synthetischen Prompts verfügbar. Führe zunächst die Verarbeitungszelle aus.")